In [64]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='west0_sales', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_sales...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [84]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [4]:
pc_pcs5_start_date = "2021-09-08"
pc_pcs5_end_date = "2021-09-30"
console_pcs5_start_date = "2021-09-16"
console_pcs5_end_date = "2021-10-20"

pgc2021_start_date = "2021-11-12"
pgc2021_end_date = "2021-12-18"

In [63]:
pc_pcs1_start_date = "2020-06-17"
pc_pcs1_end_date = "2020-07-02"
console_pcs1_start_date = "2020-06-23"
console_pcs1_end_date = "2020-07-02"

pc_pcs2_start_date = "2020-08-19"
pc_pcs2_end_date = "2020-09-08"
console_pcs2_start_date = "2020-08-27"
console_pcs2_end_date = "2020-09-10"

pc_pcs3_start_date = "2020-10-28"
pc_pcs3_end_date = "2020-11-19"
console_pcs3_start_date = "2020-10-29"
console_pcs3_end_date = "2020-11-19"

pc_pcs4_start_date = "2021-06-02"
pc_pcs4_end_date = "2021-06-24"
console_pcs4_start_date = "2021-06-10"
console_pcs4_end_date = "2021-07-14"

In [72]:
def get_pcs_paid_kpi(device, start_date, end_date):
    gcoin = pre_load_data_mart(device, start_date, end_date, "gcoin_use").where(col("event_name").like("%pcs%"))
    return gcoin.where(col("paid_use") > 0).select(countDistinct("account_id").alias("paid_pu"), sum("paid_use").alias("paid_use")).show(truncate=False)

In [74]:
get_pcs_paid_kpi("pc", pc_pcs3_start_date, pc_pcs3_end_date)

+-------+---------+
|paid_pu|paid_use |
+-------+---------+
|120313 |204457500|
+-------+---------+



In [75]:
get_pcs_paid_kpi("console", console_pcs3_start_date, console_pcs3_end_date)

+-------+--------+
|paid_pu|paid_use|
+-------+--------+
|9038   |13717270|
+-------+--------+



In [81]:
pre_load_data_mart("console", console_pcs3_start_date, console_pcs3_end_date, "gcoin_use").where(col("event_name").like("%pcs%")).select(countDistinct("account_id"), sum(col("free_use") + col("paid_use")), sum("paid_use")).show(truncate=False)

+--------------------------+--------------------------+-------------+
|count(DISTINCT account_id)|sum((free_use + paid_use))|sum(paid_use)|
+--------------------------+--------------------------+-------------+
|9990                      |17173990                  |13717270     |
+--------------------------+--------------------------+-------------+



In [76]:
get_pcs_paid_kpi("pc", pc_pcs4_start_date, pc_pcs4_end_date)

+-------+---------+
|paid_pu|paid_use |
+-------+---------+
|82231  |116746050|
+-------+---------+



In [77]:
get_pcs_paid_kpi("console", console_pcs4_start_date, console_pcs4_end_date)

+-------+--------+
|paid_pu|paid_use|
+-------+--------+
|11416  |14927960|
+-------+--------+



In [83]:
pre_load_data_mart("console", console_pcs4_end_date, console_pcs4_end_date, "user_master").where(col("lastlogindate") >= console_pcs4_start_date).select(countDistinct("accountid")).show()

+-------------------------+
|count(DISTINCT accountid)|
+-------------------------+
|                  1085232|
+-------------------------+



In [82]:
pre_load_data_mart("console", console_pcs4_start_date, console_pcs4_end_date, "gcoin_use").where(col("event_name").like("%pcs%")).select(countDistinct("account_id"), sum(col("free_use") + col("paid_use")), sum("paid_use")).show(truncate=False)

+--------------------------+--------------------------+-------------+
|count(DISTINCT account_id)|sum((free_use + paid_use))|sum(paid_use)|
+--------------------------+--------------------------+-------------+
|11982                     |16827030                  |14927960     |
+--------------------------+--------------------------+-------------+



In [5]:
pc_pcs5_gcoin = load_data_mart("pc", pc_pcs5_start_date, pc_pcs5_end_date, "gcoin_use").where(col("event_name").like("%pcs%"))
console_pcs5_gcoin = load_data_mart("console", console_pcs5_start_date, console_pcs5_end_date, "gcoin_use").where(col("event_name").like("%pcs%"))

pc_pgc2021_gcoin = load_data_mart("pc", pgc2021_start_date, pgc2021_end_date, "gcoin_use").where(col("event_name").like("%pgc%"))
console_pgc2021_gcoin = load_data_mart("console", pgc2021_start_date, pgc2021_end_date, "gcoin_use").where(col("event_name").like("%pgc%"))

In [59]:
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())
meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

In [15]:
pc_pcs5_user = load_data_mart("pc", pc_pcs5_end_date, pc_pcs5_end_date, "user_master").where(col("lastlogindate") >= pc_pcs5_start_date)
pc_pcs5_user = pc_pcs5_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
pc_pcs5_user = pc_pcs5_user.join(meta_region, pc_pcs5_user.country_new == meta_region.country_code_iso2, "left")

console_pcs5_user = load_data_mart("console", console_pcs5_end_date, console_pcs5_end_date, "user_master").where(col("lastlogindate") >= console_pcs5_start_date)
console_pcs5_user = console_pcs5_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
console_pcs5_user = console_pcs5_user.join(meta_region, console_pcs5_user.country_new == meta_region.country_code_iso2, "left")

In [27]:
pc_pgc2021_user = load_data_mart("pc", pgc2021_end_date, pgc2021_end_date, "user_master").where(col("lastlogindate") >= pgc2021_start_date)
pc_pgc2021_user = pc_pgc2021_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
pc_pgc2021_user = pc_pgc2021_user.join(meta_region, pc_pgc2021_user.country_new == meta_region.country_code_iso2, "left")

console_pgc2021_user = load_data_mart("console", pgc2021_end_date, pgc2021_end_date, "user_master").where(col("lastlogindate") >= pgc2021_start_date)
console_pgc2021_user = console_pgc2021_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
console_pgc2021_user = console_pgc2021_user.join(meta_region, console_pgc2021_user.country_new == meta_region.country_code_iso2, "left")

In [19]:
pc_pcs5_user = pc_pcs5_user.withColumnRenamed("accountid", "account_id")
pc_pcs5_gcoin = pc_pcs5_gcoin.join(pc_pcs5_user.select("account_id", "pubg_region"), "account_id")

console_pcs5_user = console_pcs5_user.withColumnRenamed("accountid", "account_id")
console_pcs5_gcoin = console_pcs5_gcoin.join(console_pcs5_user.select("account_id", "pubg_region"), "account_id")

In [28]:
pc_pgc2021_user = pc_pgc2021_user.withColumnRenamed("accountid", "account_id")
pc_pgc2021_gcoin = pc_pgc2021_gcoin.join(pc_pgc2021_user.select("account_id", "pubg_region"), "account_id")

console_pgc2021_user = console_pgc2021_user.withColumnRenamed("accountid", "account_id")
console_pgc2021_gcoin = console_pgc2021_gcoin.join(console_pgc2021_user.select("account_id", "pubg_region"), "account_id")

## PCS5

### PC

In [17]:
pc_pcs5_user.groupBy("pubg_region").agg(countDistinct("accountid").alias("au")).show(truncate=False)

+-----------+-------+
|pubg_region|au     |
+-----------+-------+
|CN         |3576753|
|NA         |134887 |
|null       |1      |
|SA         |93907  |
|KR         |1116576|
|SEA        |220259 |
|Undefined  |97     |
|JP         |23967  |
|TW/HK      |46065  |
|CIS        |217876 |
|EMEA       |374403 |
|OC         |31035  |
+-----------+-------+



In [22]:
pc_pcs5_gcoin.groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), sum(col("free_use") + col("paid_use")).alias("total_use")).show(truncate=False)

+-----------+------+---------+
|pubg_region|pu    |total_use|
+-----------+------+---------+
|CN         |128873|234727220|
|NA         |2173  |4233890  |
|SA         |1864  |3040840  |
|KR         |33547 |66805470 |
|SEA        |9090  |16424010 |
|JP         |589   |1319950  |
|TW/HK      |1291  |2536750  |
|CIS        |3160  |5090800  |
|EMEA       |6562  |12470880 |
|OC         |748   |1412660  |
+-----------+------+---------+



In [23]:
pc_pcs5_gcoin.where(col("paid_use") > 0).groupBy("pubg_region").agg(countDistinct("account_id").alias("paid_pu"), sum(col("paid_use")).alias("paid_use")).show(truncate=False)

+-----------+-------+---------+
|pubg_region|paid_pu|paid_use |
+-----------+-------+---------+
|CN         |112177 |178741490|
|NA         |2076   |3725640  |
|SA         |1609   |2205660  |
|KR         |30867  |56143030 |
|SEA        |7964   |11738140 |
|JP         |570    |1164740  |
|TW/HK      |1175   |2059520  |
|CIS        |2830   |3881230  |
|EMEA       |6174   |10537040 |
|OC         |704    |1227790  |
+-----------+-------+---------+



In [6]:
pc_pcs5_gcoin.select(sum("free_use"), sum("paid_use"), sum(col("free_use") + col("paid_use"))).show(truncate=False)

+-------------+-------------+--------------------------+
|sum(free_use)|sum(paid_use)|sum((free_use + paid_use))|
+-------------+-------------+--------------------------+
|76638190     |271424280    |348062470                 |
+-------------+-------------+--------------------------+



In [7]:
pc_pcs5_gcoin.where(col("paid_use") > 0).select(countDistinct("account_id").alias("paid_pu")).show()

+-------+
|paid_pu|
+-------+
| 166146|
+-------+



### CONSOLE

In [18]:
console_pcs5_user.groupBy("pubg_region").agg(countDistinct("accountid").alias("au")).show(truncate=False)

+-----------+------+
|pubg_region|au    |
+-----------+------+
|CN         |6294  |
|NA         |502989|
|SA         |133085|
|KR         |5857  |
|SEA        |9261  |
|Undefined  |506   |
|JP         |17520 |
|TW/HK      |4209  |
|CIS        |23143 |
|EMEA       |221459|
|OC         |28680 |
+-----------+------+



In [24]:
console_pcs5_gcoin.groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), sum(col("free_use") + col("paid_use")).alias("total_use")).show(truncate=False)

+-----------+----+---------+
|pubg_region|pu  |total_use|
+-----------+----+---------+
|CN         |48  |95120    |
|NA         |9633|12960030 |
|SA         |1275|2203680  |
|KR         |128 |226030   |
|SEA        |84  |141260   |
|JP         |481 |732650   |
|TW/HK      |66  |131330   |
|CIS        |180 |348810   |
|EMEA       |2831|4513790  |
|OC         |573 |758330   |
+-----------+----+---------+



In [25]:
console_pcs5_gcoin.where(col("paid_use") > 0).groupBy("pubg_region").agg(countDistinct("account_id").alias("paid_pu"), sum(col("paid_use")).alias("paid_use")).show(truncate=False)

+-----------+-------+--------+
|pubg_region|paid_pu|paid_use|
+-----------+-------+--------+
|CN         |47     |83720   |
|NA         |9218   |11633340|
|SA         |1244   |2026960 |
|KR         |123    |202170  |
|SEA        |82     |126870  |
|JP         |454    |641750  |
|TW/HK      |62     |113900  |
|CIS        |176    |322930  |
|EMEA       |2735   |4108680 |
|OC         |555    |689280  |
+-----------+-------+--------+



In [8]:
console_pcs5_gcoin.select(sum("free_use"), sum("paid_use"), sum(col("free_use") + col("paid_use"))).show(truncate=False)

+-------------+-------------+--------------------------+
|sum(free_use)|sum(paid_use)|sum((free_use + paid_use))|
+-------------+-------------+--------------------------+
|2161430      |19949600     |22111030                  |
+-------------+-------------+--------------------------+



In [9]:
console_pcs5_gcoin.where(col("paid_use") > 0).select(countDistinct("account_id").alias("paid_pu")).show()

+-------+
|paid_pu|
+-------+
|  14696|
+-------+



## PGC 2021

### PC

In [30]:
pc_pgc2021_user.groupBy("pubg_region").agg(countDistinct("account_id").alias("au")).show(truncate=False)

+-----------+-------+
|pubg_region|au     |
+-----------+-------+
|CN         |3759401|
|NA         |151684 |
|SA         |110431 |
|KR         |1372092|
|SEA        |249255 |
|Undefined  |172    |
|JP         |26079  |
|TW/HK      |52045  |
|CIS        |297852 |
|EMEA       |468838 |
|OC         |26975  |
+-----------+-------+



In [31]:
pc_pgc2021_gcoin.groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), sum(col("free_use") + col("paid_use")).alias("total_use")).show(truncate=False)

+-----------+------+---------+
|pubg_region|pu    |total_use|
+-----------+------+---------+
|CN         |199891|949379860|
|NA         |3159  |12718410 |
|SA         |1894  |8085310  |
|KR         |56946 |248751260|
|SEA        |9859  |47873400 |
|JP         |688   |3085020  |
|TW/HK      |1890  |8898900  |
|CIS        |4637  |17815910 |
|EMEA       |8557  |37180560 |
|OC         |888   |3831050  |
+-----------+------+---------+



In [32]:
pc_pgc2021_gcoin.where(col("paid_use") > 0).groupBy("pubg_region").agg(countDistinct("account_id").alias("paid_pu"), sum(col("paid_use")).alias("paid_use")).show(truncate=False)

+-----------+-------+---------+
|pubg_region|paid_pu|paid_use |
+-----------+-------+---------+
|CN         |189450 |659925595|
|NA         |3016   |9268630  |
|SA         |1757   |5055260  |
|KR         |54049  |179230860|
|SEA        |9486   |31373950 |
|JP         |655    |2290750  |
|TW/HK      |1796   |6498140  |
|CIS        |4268   |11640690 |
|EMEA       |8221   |25635040 |
|OC         |850    |2848150  |
+-----------+-------+---------+



In [10]:
pc_pgc2021_gcoin.select(sum("free_use"), sum("paid_use"), sum(col("free_use") + col("paid_use"))).show(truncate=False)

+-------------+-------------+--------------------------+
|sum(free_use)|sum(paid_use)|sum((free_use + paid_use))|
+-------------+-------------+--------------------------+
|403852615    |933767065    |1337619680                |
+-------------+-------------+--------------------------+



In [11]:
pc_pgc2021_gcoin.where(col("paid_use") > 0).select(countDistinct("account_id").alias("paid_pu")).show()

+-------+
|paid_pu|
+-------+
| 273548|
+-------+



### CONSOLE

In [33]:
console_pgc2021_user.groupBy("pubg_region").agg(countDistinct("account_id").alias("au")).show(truncate=False)

+-----------+------+
|pubg_region|au    |
+-----------+------+
|CN         |6502  |
|NA         |537764|
|null       |1     |
|SA         |135219|
|KR         |5543  |
|SEA        |8381  |
|Undefined  |454   |
|JP         |15807 |
|TW/HK      |3464  |
|CIS        |26849 |
|EMEA       |227671|
|OC         |24498 |
+-----------+------+



In [34]:
console_pgc2021_gcoin.groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), sum(col("free_use") + col("paid_use")).alias("total_use")).show(truncate=False)

+-----------+-----+---------+
|pubg_region|pu   |total_use|
+-----------+-----+---------+
|CN         |59   |247170   |
|NA         |10886|28239960 |
|SA         |1446 |4945550  |
|KR         |138  |531990   |
|SEA        |117  |333220   |
|JP         |564  |1780650  |
|TW/HK      |93   |386010   |
|CIS        |266  |801560   |
|EMEA       |2949 |8728890  |
|OC         |582  |1557380  |
+-----------+-----+---------+



In [35]:
console_pgc2021_gcoin.where(col("paid_use") > 0).groupBy("pubg_region").agg(countDistinct("account_id").alias("paid_pu"), sum(col("paid_use")).alias("paid_use")).show(truncate=False)

+-----------+-------+--------+
|pubg_region|paid_pu|paid_use|
+-----------+-------+--------+
|CN         |57     |165800  |
|NA         |9764   |18704690|
|SA         |1306   |3193230 |
|KR         |122    |356220  |
|SEA        |103    |219180  |
|JP         |514    |1188280 |
|TW/HK      |87     |263850  |
|CIS        |237    |536030  |
|EMEA       |2689   |5758190 |
|OC         |526    |1029810 |
+-----------+-------+--------+



In [12]:
console_pgc2021_gcoin.select(sum("free_use"), sum("paid_use"), sum(col("free_use") + col("paid_use"))).show(truncate=False)

+-------------+-------------+--------------------------+
|sum(free_use)|sum(paid_use)|sum((free_use + paid_use))|
+-------------+-------------+--------------------------+
|16137100     |31415280     |47552380                  |
+-------------+-------------+--------------------------+



In [13]:
console_pgc2021_gcoin.where(col("paid_use") > 0).select(countDistinct("account_id").alias("paid_pu")).show()

+-------+
|paid_pu|
+-------+
|  15405|
+-------+



## PCS6

In [36]:
pc_pcs6_start_date = "2022-03-16"
pc_pcs6_end_date = "2022-05-06"
console_pcs6_start_date = "2022-03-24"
console_pcs6_end_date = "2022-05-06"

In [37]:
pc_pcs6_gcoin = load_data_mart("pc", pc_pcs6_start_date, pc_pcs6_end_date, "gcoin_use").where(col("event_name").like("%pcs%"))
console_pcs6_gcoin = load_data_mart("console", console_pcs6_start_date, console_pcs6_end_date, "gcoin_use").where(col("event_name").like("%pcs%"))

In [38]:
pc_pcs6_user = load_data_mart("pc", pc_pcs6_end_date, pc_pcs6_end_date, "user_master").where(col("lastlogindate") >= pc_pcs6_start_date)
pc_pcs6_user = pc_pcs6_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
pc_pcs6_user = pc_pcs6_user.join(meta_region, pc_pcs6_user.country_new == meta_region.country_code_iso2, "left")

console_pcs6_user = load_data_mart("console", console_pcs6_end_date, console_pcs6_end_date, "user_master").where(col("lastlogindate") >= console_pcs6_start_date)
console_pcs6_user = console_pcs6_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
console_pcs6_user = console_pcs6_user.join(meta_region, console_pcs6_user.country_new == meta_region.country_code_iso2, "left")

In [40]:
pc_pcs6_user = pc_pcs6_user.withColumnRenamed("accountid", "account_id")
pc_pcs6_gcoin = pc_pcs6_gcoin.join(pc_pcs6_user.select("account_id", "pubg_region"), "account_id")

console_pcs6_user = console_pcs6_user.withColumnRenamed("accountid", "account_id")
console_pcs6_gcoin = console_pcs6_gcoin.join(console_pcs6_user.select("account_id", "pubg_region"), "account_id")

In [47]:
pc_pcs6_membership = load_membership("pubg", "pc", pc_pcs6_end_date, pc_pcs6_end_date).select("account_id", "membership").distinct()
console_pcs6_membership = load_membership("pubg", "console", console_pcs6_end_date, console_pcs6_end_date).select("account_id", "membership").distinct()

In [48]:
pc_pcs6_gcoin = pc_pcs6_gcoin.join(pc_pcs6_membership, "account_id")
console_pcs6_gcoin = console_pcs6_gcoin.join(console_pcs6_membership, "account_id")

In [51]:
pc_pcs6_user = pc_pcs6_user.join(pc_pcs6_membership, "account_id")
console_pcs6_user = console_pcs6_user.join(console_pcs6_membership, "account_id")

### PC

In [41]:
pc_pcs6_user.groupBy("pubg_region").agg(countDistinct("account_id").alias("au")).show(truncate=False)

+-----------+-------+
|pubg_region|au     |
+-----------+-------+
|CN         |8845877|
|NA         |457574 |
|null       |2      |
|SA         |421949 |
|KR         |1727283|
|SEA        |711910 |
|Undefined  |2517   |
|JP         |63022  |
|TW/HK      |123303 |
|CIS        |1204593|
|EMEA       |1407592|
|OC         |62663  |
+-----------+-------+



In [42]:
pc_pcs6_gcoin.groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), sum(col("free_use") + col("paid_use")).alias("total_use")).show(truncate=False)

+-----------+-----+---------+
|pubg_region|pu   |total_use|
+-----------+-----+---------+
|CN         |84583|134979320|
|NA         |4309 |6870730  |
|SA         |4255 |5603320  |
|KR         |25375|39829890 |
|SEA        |7144 |10171210 |
|JP         |476  |868270   |
|TW/HK      |1246 |2131520  |
|CIS        |10793|11652990 |
|EMEA       |15413|21982000 |
|OC         |824  |1417040  |
+-----------+-----+---------+



In [43]:
pc_pcs6_gcoin.where(col("paid_use") > 0).groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), sum(col("paid_use")).alias("paid_use")).show(truncate=False)

+-----------+-----+--------+
|pubg_region|pu   |paid_use|
+-----------+-----+--------+
|CN         |52879|85301235|
|NA         |2724 |4402320 |
|SA         |1800 |2204920 |
|KR         |15987|26471160|
|SEA        |3914 |5405820 |
|JP         |291  |542390  |
|TW/HK      |741  |1347020 |
|CIS        |2697 |2909391 |
|EMEA       |7602 |11139990|
|OC         |547  |947610  |
+-----------+-----+--------+



In [52]:
pc_pcs6_user.groupBy("membership").agg(countDistinct("account_id").alias("au")).show(truncate=False)

+----------+-------+
|membership|au     |
+----------+-------+
|plus      |197802 |
|basic     |7901193|
|legacy    |6854020|
+----------+-------+



In [50]:
pc_pcs6_gcoin.groupBy("membership").agg(countDistinct("account_id").alias("pu"), (sum(col("paid_use") + col("free_use"))/100).alias("total")).withColumn("arppu", col("total")/col("pu")).show(truncate=False)

+----------+------+---------+------------------+
|membership|pu    |total    |arppu             |
+----------+------+---------+------------------+
|plus      |12602 |210645.4 |16.715235676876684|
|basic     |8257  |82471.0  |9.988010173186387 |
|legacy    |133559|2061946.5|15.438469140978894|
+----------+------+---------+------------------+



In [53]:
pc_pcs6_gcoin.where(col("paid_use") > 0).groupBy("membership").agg(countDistinct("account_id").alias("paid_pu"), (sum(col("paid_use"))/100).alias("paid")).withColumn("arppu", col("paid")/col("paid_pu")).show(truncate=False)

+----------+-------+----------+------------------+
|membership|paid_pu|paid      |arppu             |
+----------+-------+----------+------------------+
|plus      |8569   |136281.0  |15.903956120900922|
|basic     |3803   |47044.6   |12.370391795950566|
|legacy    |76810  |1223392.96|15.927521937247754|
+----------+-------+----------+------------------+



### CONSOLE

In [44]:
console_pcs6_user.groupBy("pubg_region").agg(countDistinct("account_id").alias("au")).show(truncate=False)

+-----------+------+
|pubg_region|au    |
+-----------+------+
|CN         |23281 |
|NA         |904545|
|null       |3     |
|SA         |392697|
|KR         |9455  |
|SEA        |44461 |
|Undefined  |783   |
|JP         |32441 |
|TW/HK      |8900  |
|CIS        |95850 |
|EMEA       |636935|
|OC         |44418 |
+-----------+------+



In [45]:
console_pcs6_gcoin.groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), sum(col("free_use") + col("paid_use")).alias("total_use")).show(truncate=False)

+-----------+-----+---------+
|pubg_region|pu   |total_use|
+-----------+-----+---------+
|CN         |83   |140720   |
|NA         |12839|17310790 |
|SA         |2850 |3592430  |
|KR         |107  |152230   |
|SEA        |109  |156590   |
|JP         |356  |565060   |
|TW/HK      |68   |89240    |
|CIS        |281  |325000   |
|EMEA       |4729 |6432160  |
|OC         |674  |923060   |
+-----------+-----+---------+



In [46]:
console_pcs6_gcoin.where(col("paid_use") > 0).groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), sum(col("paid_use")).alias("paid_use")).show(truncate=False)

+-----------+----+--------+
|pubg_region|pu  |paid_use|
+-----------+----+--------+
|CN         |52  |90240   |
|NA         |7754|11092830|
|SA         |1268|1721870 |
|KR         |77  |108310  |
|SEA        |67  |111460  |
|JP         |274 |418290  |
|TW/HK      |55  |67200   |
|CIS        |66  |91210   |
|EMEA       |2486|3678000 |
|OC         |420 |596850  |
+-----------+----+--------+



In [54]:
console_pcs6_user.groupBy("membership").agg(countDistinct("account_id").alias("au")).show(truncate=False)

+----------+-------+
|membership|au     |
+----------+-------+
|plus      |29408  |
|basic     |1426680|
|legacy    |737681 |
+----------+-------+



In [55]:
console_pcs6_gcoin.groupBy("membership").agg(countDistinct("account_id").alias("pu"), (sum(col("paid_use") + col("free_use"))/100).alias("total")).withColumn("arppu", col("total")/col("pu")).show(truncate=False)

+----------+-----+--------+------------------+
|membership|pu   |total   |arppu             |
+----------+-----+--------+------------------+
|plus      |2429 |36355.6 |14.967311650885136|
|basic     |1403 |15934.3 |11.357305773342837|
|legacy    |18264|244582.9|13.391529785370127|
+----------+-----+--------+------------------+



In [56]:
console_pcs6_gcoin.where(col("paid_use") > 0).groupBy("membership").agg(countDistinct("account_id").alias("paid_pu"), (sum(col("paid_use"))/100).alias("paid")).withColumn("arppu", col("paid")/col("paid_pu")).show(truncate=False)

+----------+-------+--------+------------------+
|membership|paid_pu|paid    |arppu             |
+----------+-------+--------+------------------+
|plus      |1736   |25369.5 |14.613767281105991|
|basic     |1110   |13148.1 |11.845135135135136|
|legacy    |9673   |141245.0|14.601984906440608|
+----------+-------+--------+------------------+

